浪漫派
李白 161-185

田園山水派
王維 125-128
孟浩然 159-160
柳宗元 350-353

社會寫實派
杜甫 216-234

邊塞派
岑參 198-201
高適 211-214
王昌齡 140-143

In [3]:
# plain data collect
import requests as rq
from bs4 import BeautifulSoup
import re
import csv
base_url = "https://ctext.org/quantangshi/"
folder = "plain_data/"
file_path = ["浪漫派.csv", "山水田園派.csv", "社會寫實派.csv", "邊塞派.csv"]
books = [[[161, 185]], [[125, 128], [159, 160], [350, 353]], [[216, 234]]
         , [[198, 201], [211, 214], [140, 143]]]
for file_itr in range(4):
    with open (folder+file_path[file_itr], 'w+', newline='', encoding='utf-8-sig') as csvfile:
        cnt = 0
        csvwriter = csv.writer(csvfile)
        for j in range(len(books[file_itr])):
            for i in range(books[file_itr][j][0], books[file_itr][j][1]+1):
                rsp_web = rq.get(base_url+str(i))
                soup = BeautifulSoup(rsp_web.text, 'html.parser')
                content = soup.find('div', id = 'content3')
                ctexts = content.select('.ctext:not([class*=" "])')
                for ctext in ctexts:
                    if ctext.get_text().strip() == '':
                        continue
                    text = ctext.get_text().replace('\n','')
                    lines = re.split('[。?]', text)
                    for line in lines:
                        if line.strip()=='' or re.compile(r'[^\u4e00-\u9fff]').match(line):
                            continue
                        csvwriter.writerow([line])
                        cnt+=1
        print(folder+file_path[file_itr]+" done\ndata size: "+ str(cnt))

plain_data/浪漫派.csv done
data size: 6759
plain_data/山水田園派.csv done
data size: 4526
plain_data/社會寫實派.csv done
data size: 9664
plain_data/邊塞派.csv done
data size: 4761


In [3]:
#data balance and augmentation
import random
import csv
def data_augmentation(input_path, output_path):
    with open (input_path, 'r', newline='', encoding='utf-8-sig') as inputfile, \
         open(output_path, 'w', newline='', encoding='utf-8-sig') as outputfile:
        cnt = 0
        reader = csv.reader(inputfile)
        writer = csv.writer(outputfile)
        data = list(reader)
        rate = 9000/len(data)
        for row in data:
            writer.writerows([row])
            cnt+=1
        if(rate>1):
            sample = random.sample(data, int(len(data)*(rate-1)))
            for row in sample:
                segments = row[0].split("，")
                if(len(segments)>=2):
                    s = "{}，{}".format(segments[1],segments[0])
                    writer.writerow([s])
                    cnt+=1
        print(cnt)
folder1 = "plain_data/"
folder2 = "proccessed_data/"
files = ["浪漫派.csv", "山水田園派.csv", "社會寫實派.csv", "邊塞派.csv"]
for i in range(4):
    data_augmentation(folder1+files[i], folder2+files[i])


8999
8988
9664
8997


In [1]:
# train/text data collecting
import os
import csv

folder_path = 'plain_data/'
train = open('./data/train.csv', 'w+', encoding='utf-8-sig')
test = open('./data/test.csv', 'w+', encoding='utf-8-sig')
all = open('./data/all.csv', 'w+', encoding='utf-8-sig')
files = os.listdir(folder_path)

# 四大分類
dict = {"浪漫派":1, "山水田園派":2, "社會寫實派":3, "邊塞派":4}

train_cnt = 0
test_cnt = 0

for file in files:
    cnt = 0
    file_path = os.path.join(folder_path, file)
    # print(file[:-4])
    with open(file_path, 'r', encoding = 'utf-8-sig') as f:
        csvreader = csv.reader(f)
        for row in csvreader:
            text = ' '.join(row[0])
            label = dict[file[:-4]]
            data = "{}, {}".format(label, text)
            all.write("{}, {}\n".format(label, row[0]))
            if(cnt%10==0):
                test.write(data+'\n')
                test_cnt+=1
            else:
                train.write(data+'\n')
                train_cnt+=1
            cnt+=1
print("train data cnt: {}\ntest data cnt: {}".format(train_cnt, test_cnt))
print("total data cnt: {}".format(train_cnt+test_cnt))

train data cnt: 23137
test data cnt: 2573
total data cnt: 25710
